In [ ]:
import pandas as pd
import numpy as np


In [ ]:
#load patient data
df = pd.read_csv("../docs/MSDS_cleaned_0122.csv")
df.info()

#### 1. Calculation of Composite Score

The parameters used in computing the composite score are the GAP score, L1PA penalty, L4SI penalty, T4L1PA penalty, LL penalty, and improvement in GAP category (P, MD, SD). The parameters are given equal weights. 

Calculation method:

GAP: The GAP score (1-13) is converted to a 0-100 scale.

Penalties: Quadratic penalties are calculated for violation of the respective constraints and normalized based on the maximum violation in the dataset. Then the (weighted) penalties are added to the composite score.

Improvement in GAP (Option 2 in calculation doc): 

No penalty is applied if the patient's condition improved from severely disproportioned or moderately disproportioned to proportioned. Otherwise, a penalty of 30 is applied if the patient's condition improved from severely disproportioned to moderately disproportioned. A penalty of 100 is added if the patient's condition remains in the same category or worsens. 

* note : may need to find a better way to normalize

In [ ]:
#create a score system for gap change based on change in gap category - option 1
df["gap_category_postop"] = pd.cut(df["gap_score_postop"], bins=[-np.inf, 2, 6, 13], labels=["P", "MD", "SD"])


def composite_score_calc(df, w1=1, w2=1, w3=1, w4=1, w5=1, w6=1):
    """
    Compute composite score based on gap score and quadartic penalties for constraint violations.

    Parameters:
    df: patient surgical data in Pandas DataFrame
    w1: weight for L1PA penalty
    w2: weight for L4S1 penalty
    w3: weight for T4L1PA penalty
    w4: weight for LL penalty
    w5: weight for gap improvement score
    multiplier : scaling factor for penalties (since gap score is on a different scale)

   Returns:
   composite scores 
    """
    #calculate relative weights
    weights = [w1, w2, w3, w4, w5, w6]
    total_weight = sum(weights)
    rel_weights = [w / total_weight for w in weights]

    # convert gap scores to numpy array for simpler calculations
    gap_score_postop = np.array(df["gap_score_postop"] * 100/13)
   
    # calculate constraint penalties

    # 1) L1PA quadratic penalty
    l1pa_pen = [0 if abs(i) <= 3 else (abs(i)-3)**2 for i in df["L1PA_ideal_mismatch_postop"]]

    #normalize L1PA penalty
    l1pa_pen = np.array(l1pa_pen) / max(l1pa_pen) * 100

    # 2) L4S1 quadratic penalty
    l4s1_pen = [0 if 35 <= i <= 45 else (35 - i) ** 2 if i < 35 else (i - 45) ** 2 for i in df["L4_S1_postop"]]
   
    #normalize L4S1 penalty
    l4s1_pen = np.array(l4s1_pen) / max(l4s1_pen) * 100

    # 3) T4L1PA penalty
    t4l1pa_pen = [0 if abs(i) <= 3 else (abs(i)-3)**2 for i in df["T4L1PA_ideal_mismatch_postop"]]
    
    #normalize T4L1PA penalty
    t4l1pa_pen = np.array(t4l1pa_pen)/ max(t4l1pa_pen) * 100
 
    # 4) LL penalty
    ll_ideal = 0.54 * df["PI_preop"] + 27.6
    ll_mismatch = df["LL_postop"] - ll_ideal
    ll_pen = [0 if abs(i) <= 3 else (abs(i) - 3) ** 2 for i in ll_mismatch]
  
    #normalize LL penalty 
    ll_pen = (np.array(ll_pen)) / max(ll_pen) * 100

    #gap improvement score based on category change

    gap_category_improvement = []
    for i in range(len(df)):
        if df["gap_category_postop"][i] =="P" and (df["gap_category"][i]) in ["SD","MD", "P"]:
            gap_category_improvement.append(0)
        elif df["gap_category_postop"][i] =="MD" and df["gap_category"][i] == "SD":
            gap_category_improvement.append(30)
        else:
            gap_category_improvement.append(100)
    gap_category_improvement = np.array(gap_category_improvement)

    # Composite score

    composite = (rel_weights[0] * gap_score_postop +
                rel_weights[1] * l1pa_pen +
                rel_weights[2] * l4s1_pen +
                rel_weights[3] * t4l1pa_pen +
                rel_weights[4] * ll_pen +
                rel_weights[5] * gap_category_improvement)
    
    return composite


composite_scores = composite_score_calc(df) #weights can be adjusted here

print(composite_scores)

##### 2. Add Composite Scores to Data Frame

In [ ]:
#show all rows
pd.set_option("display.max_rows", None)

df["composite_scores"] = pd.DataFrame(composite_scores)
print(df[["gap_score_postop","composite_scores"]])